In [1]:
import json
import pandas as pd
import urllib.request
import urllib.parse
import urllib.error
import os
from IPython.display import Image
import numpy as np
import cv2
import math

# 📂 Import Dataset

In [3]:
'''
1. Dataframe for train
'''

DataSet = pd.read_csv("/home/kannika/codes_AI/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-Dataset-yes-Dropdup.csv")
print("subclass: ", set(DataSet["subclass"]))
print("Source: ", set(DataSet["folder_source"]))
print("Complete: ", set(DataSet["complete"]))
print(DataSet.shape)
DataSet.head()

subclass:  {'Normal', 'No_Splenectomy', 'Splenectomy'}
Source:  {'SSD_Backup'}
Complete:  {'yes'}
(1076538, 9)


,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,complete,Code,subclass
0,0,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy
1,1,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy
2,2,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy
3,3,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy
4,4,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy


In [4]:
'''
2. DataFrame for summarize cap code&classes
'''

df_patien = pd.read_csv("/home/kannika/codes_AI/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-SummarizeFolder.csv")
print(df_patien.shape)
df_patien.head()

(53, 4)


,Unnamed: 0,classes,Code,NumFolder
0,0,No_Splenectomy,HN38,12
1,1,No_Splenectomy,HN18,8
2,2,No_Splenectomy,HN33,8
3,3,No_Splenectomy,HN35,8
4,4,No_Splenectomy,HN22,12


## 💾 Split 6-Fold

In [48]:
import random

lst_sheet = list(set(df_patien["classes"]))
#print(len(lst_sheet))
print(lst_sheet)
n = 3
DF_fold = pd.DataFrame(columns=list(df_patien.columns))

for j in range(len(lst_sheet)):
    df_classes = df_patien[df_patien["classes"]==lst_sheet[j]]
    print(f"[INFO]: [{j+1}].  For Classes {lst_sheet[j]} With Shape ==> {df_classes.shape}")
    for i in range(1, 7):
        df_classes_ran = df_classes.sample(n, replace=False)
        df_classes.drop(index=df_classes_ran.index, inplace=True)
        #df_patien.drop(index=df_classes_ran.index, inplace=True)
        #random.shuffle(lst_fold)
        df_classes_ran["fold"] = i
        DF_fold = DF_fold.append(df_classes_ran)
        if df_classes.shape[0] < 3:
            #print(df_classes.shape[0])
            break
    df_classes["fold"] = 6
    DF_fold = DF_fold.append(df_classes)

['No_Splenectomy', 'Splenectomy', 'Normal']
[INFO]: [1].  For Classes No_Splenectomy With Shape ==> (17, 4)
[INFO]: [2].  For Classes Splenectomy With Shape ==> (18, 4)
[INFO]: [3].  For Classes Normal With Shape ==> (18, 4)


/tmp/ipykernel_3033/1428967412.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes.drop(index=df_classes_ran.index, inplace=True)
/tmp/ipykernel_3033/1428967412.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  DF_fold = DF_fold.append(df_classes_ran)
/tmp/ipykernel_3033/1428967412.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_classes.drop(index=df_classes_ran.index, inplace=True)
/tmp/ipykernel_3033/1428967412.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [50]:
DF_fold = DF_fold.reset_index(drop=True)
print(DF_fold.shape)
DF_fold['fold'] = DF_fold['fold'].astype(int)
DF_fold.head()

(53, 5)


,Unnamed: 0,classes,Code,NumFolder,fold
0,0,No_Splenectomy,HN38,12,1
1,3,No_Splenectomy,HN35,8,1
2,11,No_Splenectomy,HN43,8,1
3,2,No_Splenectomy,HN33,8,2
4,13,No_Splenectomy,HN34,8,2


In [54]:
DF_fold['fold'].isnull().sum()

0

In [51]:
## Check
for j in range(len(lst_sheet)):
    df_classes = DF_fold[DF_fold["classes"]==lst_sheet[j]]
    print(f"[INFO]: [{j+1}].  For Classes {lst_sheet[j]} With Shape ==> {df_classes.shape}")
    print("Check 6-Fold")
    lst_fold = list(set(df_classes["fold"]))
    print(f"Number of Fold {len(lst_fold)} ==> {lst_fold}")
    for F in lst_fold:
        df_f =  df_classes[df_classes["fold"]==F]
        print(f"[INFO]: FOR Fold {F} ==> With {df_f.shape[0]} Person!!")
    print("="*125)

[INFO]: [1].  For Classes No_Splenectomy With Shape ==> (17, 5)
Check 6-Fold
Number of Fold 6 ==> [1, 2, 3, 4, 5, 6]
[INFO]: FOR Fold 1 ==> With 3 Person!!
[INFO]: FOR Fold 2 ==> With 3 Person!!
[INFO]: FOR Fold 3 ==> With 3 Person!!
[INFO]: FOR Fold 4 ==> With 3 Person!!
[INFO]: FOR Fold 5 ==> With 3 Person!!
[INFO]: FOR Fold 6 ==> With 2 Person!!
[INFO]: [2].  For Classes Splenectomy With Shape ==> (18, 5)
Check 6-Fold
Number of Fold 6 ==> [1, 2, 3, 4, 5, 6]
[INFO]: FOR Fold 1 ==> With 3 Person!!
[INFO]: FOR Fold 2 ==> With 3 Person!!
[INFO]: FOR Fold 3 ==> With 3 Person!!
[INFO]: FOR Fold 4 ==> With 3 Person!!
[INFO]: FOR Fold 5 ==> With 3 Person!!
[INFO]: FOR Fold 6 ==> With 3 Person!!
[INFO]: [3].  For Classes Normal With Shape ==> (18, 5)
Check 6-Fold
Number of Fold 6 ==> [1, 2, 3, 4, 5, 6]
[INFO]: FOR Fold 1 ==> With 3 Person!!
[INFO]: FOR Fold 2 ==> With 3 Person!!
[INFO]: FOR Fold 3 ==> With 3 Person!!
[INFO]: FOR Fold 4 ==> With 3 Person!!
[INFO]: FOR Fold 5 ==> With 3 Person

In [52]:
## Check
lst_fold = list(set(DF_fold["fold"]))

for j in range(len(lst_fold)):
    df_classes = DF_fold[DF_fold["fold"]==lst_fold[j]]
    print(f"[INFO]: [{j+1}].  For fold {lst_fold[j]} With Shape ==> {df_classes.shape}")
    print("Check 6-Fold")
    lst_person = list(set(df_classes["Code"]))
    print(f"[INFO]: {len(lst_person)} ==> PerSon!!!")
    print("="*125)

[INFO]: [1].  For fold 1 With Shape ==> (9, 5)
Check 6-Fold
[INFO]: 9 ==> PerSon!!!
[INFO]: [2].  For fold 2 With Shape ==> (9, 5)
Check 6-Fold
[INFO]: 9 ==> PerSon!!!
[INFO]: [3].  For fold 3 With Shape ==> (9, 5)
Check 6-Fold
[INFO]: 9 ==> PerSon!!!
[INFO]: [4].  For fold 4 With Shape ==> (9, 5)
Check 6-Fold
[INFO]: 9 ==> PerSon!!!
[INFO]: [5].  For fold 5 With Shape ==> (9, 5)
Check 6-Fold
[INFO]: 9 ==> PerSon!!!
[INFO]: [6].  For fold 6 With Shape ==> (8, 5)
Check 6-Fold
[INFO]: 8 ==> PerSon!!!


--------------------------------------------------------------------------------------------------------------------------------

In [53]:
DF_fold.to_csv("/home/kannika/codes_AI/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-SummarizeFolder_Fold.csv")

--------------------------------------------------------------------------------------------------------------------------------

# Defind Fold in DataFrame

In [55]:
## 1. Create Dict
lst_Code = DF_fold["Code"].tolist()
lst_fold = DF_fold["fold"].tolist()
dict_code = zip(lst_Code, lst_fold)
output_dict = dict(dict_code)
print(output_dict)

{'HN38': 1, 'HN35': 1, 'HN43': 1, 'HN33': 2, 'HN34': 2, 'HN14': 2, 'HN21': 3, 'HN20': 3, 'HN22': 3, 'HN41': 4, 'HN42': 4, 'HN30': 4, 'HN31': 5, 'HN18': 5, 'HN15': 5, 'HN29': 6, 'HN26': 6, 'HN17': 1, 'HN13': 1, 'HN12': 1, 'HN32': 2, 'HN19': 2, 'HN28': 2, 'HN37': 3, 'HN25': 3, 'HN02': 3, 'HN24': 4, 'HN40': 4, 'HN27': 4, 'HN16': 5, 'HN36': 5, 'HN44': 5, 'HN01': 6, 'HN39': 6, 'HN23': 6, 'NBL15': 1, 'NBL14': 1, 'NBL23': 1, 'NBL01': 2, 'NBL00': 2, 'NBL19': 2, 'NBL20': 3, 'NBL21': 3, 'NBL16': 3, 'NBL10': 4, 'NBL08': 4, 'NBL13': 4, 'NBL11': 5, 'NBL22': 5, 'NBL12': 5, 'NBL09': 6, 'NBL18': 6, 'NBL17': 6}


In [56]:
## 2. Map fold
DataSet["fold"]= DataSet['Code'].map(output_dict)
print(DataSet.shape)
DataSet.head()

(1076538, 10)


,Unnamed: 0,image_name,image_path,image_folder,folder_source,classes,complete,Code,subclass,fold
0,0,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy,2
1,1,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy,2
2,2,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy,2
3,3,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy,2
4,4,HN34_D0_30HZ_20XINF_UWELL_25660131_155730_0000...,/media/HDD/rheology2023/Blood_Viscosity/SSD_Ba...,HN34_D0_30HZ_20XINF_UWELL_25660131_155730,SSD_Backup,HN,yes,HN34,No_Splenectomy,2


In [57]:
DataSet['fold'].isnull().sum()

0

In [58]:
set(DataSet['fold'])

{1, 2, 3, 4, 5, 6}

In [79]:
## Check
def _(DataSet):
    lst_fold = list(set(DataSet["fold"]))
    for j in range(len(lst_sheet)):
        df_classes = DataSet[DataSet["subclass"]==lst_sheet[j]]
        Class_folder = list(set(df_classes["image_folder"]))
        print(f"[INFO]: [{j+1}].  For Classes {lst_sheet[j]} With Shape ==> {len(Class_folder)} Folder, {df_classes.shape}")
        print("Check 6-Fold")
        lst_fold = list(set(df_classes["fold"]))
        print(f"Number of Fold {len(lst_fold)} ==> {lst_fold}")
        for F in lst_fold:
            df_f =  df_classes[df_classes["fold"]==F]
            list_folder = list(set(df_f["image_folder"]))
            list_code =  list(set(df_f["Code"]))
            print(f"[INFO]: FOR Fold {F} ==> With {len(list_folder)} Folder, {df_f.shape[0]} images!! ==> {len(list_code)} PerSon!!")
        print("="*125)
    
    return

In [80]:
_(DataSet)

[INFO]: [1].  For Classes No_Splenectomy With Shape ==> 152 Folder, (304152, 10)
Check 6-Fold
Number of Fold 6 ==> [1, 2, 3, 4, 5, 6]
[INFO]: FOR Fold 1 ==> With 28 Folder, 56028 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 2 ==> With 24 Folder, 48024 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 3 ==> With 28 Folder, 56028 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 4 ==> With 32 Folder, 64032 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 5 ==> With 24 Folder, 48024 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 6 ==> With 16 Folder, 32016 images!! ==> 2 PerSon!!
[INFO]: [2].  For Classes Splenectomy With Shape ==> 185 Folder, (370185, 10)
Check 6-Fold
Number of Fold 6 ==> [1, 2, 3, 4, 5, 6]
[INFO]: FOR Fold 1 ==> With 32 Folder, 64032 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 2 ==> With 24 Folder, 48024 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 3 ==> With 26 Folder, 52026 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 4 ==> With 28 Folder, 56028 images!! ==> 3 PerSon!!
[INFO]: FOR Fold 5 ==> With 28 Folder, 56028 

In [66]:
304152+370185+402201

1076538

In [81]:
1076538/29

37122.0

In [76]:
DataSet.to_csv("/home/kannika/codes_AI/Rheology_Blood/Dataset_Rheology_Blood_Viscosity_HN_NBL-Dataset-6Fold.csv")